# EXTRACCIÓN DE LOS DATOS DE TWITTER

Para la red social de Twitter, se extraerán datos con dos scripts diferentes.<br>
- Script para sacar información de los perfiles e introducirlos en la BBDD
- Script para sacar información de los tweets e introducirlos en la BBDD

## PSEUDOCÓDIGO

**El psudocódigo del programa será de la siguiente manera**

Abrir Base de Datos
Seleccionar y extraer los datos de la tabla donde se almacenan los nombres de las diferentes cuentas de Twitter<br><br>
Extraer las cuentas de twitter<br><br>
Para cada una de las cuentas:<br>
   &emsp; extraer los datos de la cuentas<br>
   &emsp; introducir los datos en la tabla correspondiente<br><br>
   &emsp; Para cada uno de los tweets de la cuenta:<br>
   &emsp; &emsp; extraer los datos de los tweets<br>
   &emsp; &emsp; introducir los datos en la tabla correspondiente<br>

## LIBRERÍAS

Para la correcta extracción y recopilación de los datos, se aplicarán diferentes librerías pero la librería principal es:<br><br>
**__SNSCRAPE__**: Librería desarrollada por el usuario de Github JustAnotherArchivist que tiene por misión facilitar la extracción de datos de diferentes Redes Sociales. En este caso en concreto se utilizará el módulo de Twitter, que se puede consultar __[aquí](https://github.com/JustAnotherArchivist/snscrape/blob/master/snscrape/modules/twitter.py)__. Como no tiene documentación, hay que leer y entender el código para su posterior aplicación.<br><br>

Otras librerías que se usan son:
- **Pandas** : Manipulación de datos
- **Mysql.connector**: Conexión con la base de datos
- **Codecs**: Normalización de caracteres


## PASOS PREVIOS

Para realizar estos scripts, primeramente se añaden dos tablas a la BBDD con la siguiente sintaxis:

- **TABLA PARA PERFILES DE TWITTER**:

CREATE TABLE IF NOT EXISTS perfiles_twitter(<br>
    &emsp; Universidad VARCHAR(100),<br>
	&emsp; Abreviatura VARCHAR(20),<br>
    &emsp; Perfil VARCHAR(200),<br>
    &emsp; Creacion DATETIME,<br>
    &emsp; Verificado BOOL,<br>
    &emsp; Descripcion TEXT,<br>
    &emsp; Seguidores BIGINT,<br>
    &emsp; Seguidos BIGINT,<br>
    &emsp; Favoritos BIGINT,<br>
    &emsp; Media BIGINT,<br>
    &emsp; Localizacion VARCHAR(200),<br>
    &emsp; PRIMARY KEY(Universidad)<br>
)ENGINE = INNODB;<br>


- **TABLA PARA LOS TWEETS**:

CREATE TABLE IF NOT EXISTS tweets(<br>
	&emsp; id_tweet BIGINT,<br>
    &emsp; universidad VARCHAR(100),<br>
    &emsp; fecha TIMESTAMP,<br>
    &emsp; contenido TEXT,<br>
    &emsp; contenido_raw TEXT,<br>
    &emsp; replicas INT,<br>
    &emsp; retweet INT,<br>
    &emsp; me_gusta INT,<br>
    &emsp; rt_mg INT,<br>
    &emsp; conversationID BIGINT,<br>
    &emsp; idioma VARCHAR(4),<br>
    &emsp; dispositivo VARCHAR(50),<br>
    &emsp; Es_Retweet VARCHAR(280),<br>
    &emsp; Es_Quoted TINYINT,<br>
    &emsp; Usuarios_mencionados INT,<br>
    &emsp; PRIMARY KEY(id_tweet)<br>
)ENGINE = INNODB;<br>


## CÓDIGO

Ya que se tiene todo los requisitos, ahora se abre la conexión con la Base de Datos y se empieza a codificar. <br>
Lo primero de todo es importar las librerias que necesitamos

In [ ]:
pip install snscrape

In [ ]:
import snscrape.modules.twitter as snst
import pandas as pd
import mysql.connector
import codecs

Acto seguido, se abre la conexión de la base de Datos con Python 

In [ ]:
database = mysql.connector.connect(
    host= "localhost",
    user= "root",
    passwd= "",
    database= "TFG",
    use_unicode = True, 
    charset="utf8mb4",
    collation = "utf8mb4_general_ci"
)
database.set_charset_collation("utf8mb4","utf8mb4_general_ci")

cursor= database.cursor()

Se extraen las cuentas de Twitter de la tabla de la Base de Datos **"universidades"**

In [ ]:
datos = pd.read_sql("SELECT * FROM universidades",database)
cuentas_twitter = datos["Twitter"].tolist()
print(datos.info())

Como todas las Universidades tienen cuenta de Twitter, nos ahorramos la limpieza intermedia.<br>
Se pasa a sacar la información.<br>
Para hacerlo más legible, se harán 2 bucles. Uno con los perfiles y otro con los tweets.

In [ ]:
## EXTRACCIÓN Y PROCESAMIENTO DE DATOS DE PERFILES ##
for b in range(len(cuentas_twitter)):
    for perfil in snst.TwitterUserScraper(cuentas_twitter[b]).get_items():
        print(perfil.user.username)
        twitter_data = [universidad[b],
                        abreviaturas[b],
                        perfil.user.username,
                        perfil.user.created,
                        perfil.user.verified,
                        perfil.user.description,
                        perfil.user.followersCount,
                        perfil.user.friendsCount,
                        perfil.user.favouritesCount,
                        perfil.user.mediaCount,
                        perfil.user.location]
        twitter_data[3] = twitter_data[3].strftime('%Y-%m-%d %H:%M:%S')
        twitter_data = tuple(twitter_data)
        print(len(twitter_data))
        print(twitter_data)
        cursor.execute("INSERT INTO perfiles_twitter VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",twitter_data)
        database.commit()
        

In [ ]:
## EXTRACCIÓN Y PROCESAMIENTO DE DATOS DE TWEETS##
for b in range(len(cuentas_twitter)): 
    print(cuentas_twitter[b])
    for i,tweet in enumerate(snst.TwitterUserScraper(cuentas_twitter[b]).get_items()):
        datos_tweets = [tweet.id,
        tweet.user.username,
        tweet.date,
        tweet.content,
        tweet.renderedContent,
        tweet.replyCount,
        tweet.retweetCount,
        tweet.likeCount,
        tweet.quoteCount,
        tweet.conversationId,
        tweet.lang,
        tweet.source,
        tweet.retweetedTweet,
        tweet.quotedTweet,
        tweet.mentionedUsers
        ]
        datos_tweets[2] = datos_tweets[2].strftime('%Y-%m-%d %H:%M:%S')
        datos_tweets[11] = datos_tweets[11][datos_tweets[11].find(">")+1: datos_tweets[11].rfind("<")]
        if type(datos_tweets[-1]) == list:
            datos_tweets[-1] = len(datos_tweets[-1])
        else:
            datos_tweets[-1] = 0

        if type(datos_tweets[-2]) != None:
            datos_tweets[-2] = True
        else:
            datos_tweets[-2] = False

        datos_tweets = tuple(datos_tweets)
        cursor.execute("INSERT INTO tweets VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)",datos_tweets)
        database.commit()

### Con este pequeño script están los datos de la red social Twitter extraidos.<br><br> Duración aproximada 8h